## 学生抑郁分析

**目录**
1. [探索性数据分析（EDA）](#1-探索性数据分析-eda)
2. [聚类分析](#2-聚类分析)
3. [分类模型与预测分析](#3-分类模型与预测分析)
4. [关联规则挖掘](#4-关联规则挖掘)


### 导入第三方库

In [ ]:
import numpy as np


### 1. 探索性数据分析 (EDA)

### 2. 聚类分析

### 3. 分类模型与预测分析

### 4. 关联规则挖掘